# Torchtext

useful links: <br>
    https://pytorch.org/text/data.html#field<br>
    https://www.kaggle.com/maxl28618/toxic-comments-lstm-in-pytorch-with-torchtext/notebook<br>
    https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f<br>
    https://medium.com/@sonicboom8/sentiment-analysis-torchtext-55fb57b1fab8<br>
    

In [1]:
#libraries
import pickle
from collections import Counter
import sys
import csv

In [37]:
# Libraries
import time
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchtext
import numpy as np
# Preliminaries
from sklearn.model_selection import train_test_split
from torchtext.data import Field, TabularDataset, BucketIterator

# Models

import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F

# Training

import torch.optim as optim

# Evaluation
import sklearn
from sklearn.metrics import f1_score, precision_score, recall_score,fbeta_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error,mean_absolute_error

import seaborn as sns

#from pytorch_lightning import metrics
#from pytorch_lightning.metrics import F1
#from pytorch_lightning.metrics import Precision
#from pytorch_lightning.metrics import Recall
#from pytorch_lightning.metrics import MeanAbsoluteError
#from pytorch_lightning.metrics import MeanSquaredError
#from pytorch_lightning.metrics import ConfusionMatrix


In [9]:
#!poetry add torchtext#torchtext==0.4

In [ ]:
import logging
logging.basicConfig(filename='app.log', filemode='w', format='%(asctime)s,%(msecs)d %(name)s - %(levelname)s - %(message)s',datefmt='%H:%M:%S', level=logging.DEBUG)

In [ ]:
# just for Colab
#csv.field_size_limit(sys.maxsize)
csv.field_size_limit(min(sys.maxsize, 2147483646))

In [ ]:
destination_folder='/data/'
#'/content/drive/MyDrive/gene_calling/'

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


/nfs/production/panda/ensembl/ftricomi/.pyenv/versions/3.8.6/envs/gene-finder/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [17]:
print(f"{torch.__version__}")
print(f"{torch.version.cuda}")
print(f"{torch.backends.cudnn.enabled}")
print(f"{torch.cuda.is_available()}")
print(f"{device}")
if torch.cuda.is_available():
        print(f"{torch.cuda.get_device_properties(device)}")

1.7.0
10.2
True
False
cpu


## prepare dataset

In [3]:
def load_pickle(path):
    with open(path, "rb") as fh:
        data = pickle.load(fh)
    return data  

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/gene_calling/word_df_data1.csv')

In [4]:
data=load_pickle("data/word_df_metrics.pkl")

In [5]:
data.head()

,encoding,genomic_sequence,sequence_len,sequence_splitted,word_count,vocabulary,lexical density
0,O,GGATCCGTTCGAAACAGGTTAGCCTACTATAATATAAGGATTGGAT...,11917,GGA TCC GTT CGA AAC AGG TTA GCC TAC TAT AAT AT...,3973,65,0.016360
1,S,ATG,3,ATG,1,1,1.000000
2,C,TTTCAGTTCGCTAAGTTTTCAAAGTCCAAAGAGCGCAGACTAGCCA...,318,TTT CAG TTC GCT AAG TTT TCA AAG TCC AAA GAG CG...,106,46,0.433962
3,E,TAA,3,TAA,1,1,1.000000
4,O,AGGCAAAGAAAAAAGGGATCCGCCTCGAATCAAAACGTTCTTTCTT...,4614,AGG CAA AGA AAA AAG GGA TCC GCC TCG AAT CAA AA...,1538,64,0.041612


In [6]:
#encode label 
label_encoder = {l: i for i, l in enumerate(data['encoding'].unique())}
data['encoding'] = data['encoding'].apply(lambda y: label_encoder[y])
label_encoder 

{'O': 0, 'S': 1, 'C': 2, 'E': 3, 'J': 4, 'I': 5}

In [7]:
s=(data['sequence_splitted'].apply(lambda x: len((str(x).split())))) # counts the number of words for  max_length
s.describe()

count    1.155759e+06
mean     3.150967e+02
std      5.127760e+03
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      4.500000e+01
max      1.666670e+05
Name: sequence_splitted, dtype: float64

In [8]:
subset_0_data=data[data['encoding']==0]
subset_1_data=data[data['encoding']==1]
subset_2_data=data[data['encoding']==2]
subset_3_data=data[data['encoding']==3]
subset_4_data=data[data['encoding']==4]
subset_5_data=data[data['encoding']==5]

In [9]:
#balanced dataset
df_subset=subset_0_data[0:2000]
df_subset=df_subset.append(subset_1_data[0:2000],ignore_index=True)
df_subset=df_subset.append(subset_2_data[0:2000],ignore_index=True)
df_subset=df_subset.append(subset_3_data[0:2000],ignore_index=True)
df_subset=df_subset.append(subset_4_data[0:2000],ignore_index=True)
df_subset=df_subset.append(subset_5_data[0:2000],ignore_index=True)

In [10]:
len(df_subset)

12000

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_subset['sequence_splitted'], df_subset['encoding'], test_size=0.15, random_state=1, stratify=df_subset['encoding'])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1, stratify=y_train)

X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
X_val.reset_index(inplace=True, drop=True)
y_val.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [12]:
len(X_train),len(X_val),len(X_test)

(8160, 2040, 1800)

In [13]:
df_train=pd.DataFrame()
df_train['label']=y_train
df_train['sequence_splitted']=X_train

df_valid=pd.DataFrame()
df_valid['label']=y_val
df_valid['sequence_splitted']=X_val

df_test=pd.DataFrame()
df_test['label']=y_test
df_test['sequence_splitted']=X_test

df_train.head()

,label,sequence_splitted
0,0,TGC AAC ACA GAG CAC CCA TCT CTC TCC TTC AAC TC...
1,4,AAG
2,1,ATG
3,4,TAG
4,5,AAG TGC CTT TTT TTT CAA ACA TTT CTT TCA AGT CT...


In [15]:
# Write preprocessed data
df_train.to_csv('data/train2.csv', index=False)
df_valid.to_csv( 'data/valid2.csv', index=False)
df_test.to_csv( 'data/test2.csv', index=False)

## Pre-processing with Torchtext

In [ ]:
def load_fields():
    # Fields for encoding
    tokenize = lambda x: x.split(' ')

    text_field = Field(sequential=True, tokenize=tokenize,lower=False, include_lengths=True, batch_first=True,pad_token='O')
    label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)

    #same order and same column name as in the dataframe 
    fields = [ ('label', label_field),('sequence_splitted', text_field)]

    return text_field, label_field, fields

def load_tabular_dataset(fields):    
    #train='train.csv', validation='valid.csv', test='test.csv',
    train, valid, test = TabularDataset.splits(path='/content/drive/MyDrive/gene_calling/', train='train2.csv', validation='valid2.csv', test='test2.csv',
                                                format='CSV', fields=fields, skip_header=True)
    return train, valid, test

    # Iterators
def load_iterators(train, valid, test):   
    
    #each batch is of type torch.LongTensor, they are the numericalized batch
    train_iter = BucketIterator(train, batch_size=32, sort_key=lambda x: len(x.sequence_splitted),
                                device=device, sort=True, sort_within_batch=True)
    valid_iter = BucketIterator(valid, batch_size=32, sort_key=lambda x: len(x.sequence_splitted),
                                device=device, sort=True, sort_within_batch=True)
    test_iter = BucketIterator(test, batch_size=32, sort_key=lambda x: len(x.sequence_splitted),
                                device=device, sort=True, sort_within_batch=True)

    return train_iter, valid_iter, test_iter

In [18]:
# Fields for encoding
tokenize = lambda x: x.split(' ')

text_field = Field(sequential=True, tokenize=tokenize,lower=False, include_lengths=True, batch_first=True,pad_token='O')
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)

#use_vocab=False have preprocessed the label to be integers
#TorchText will dynamically pad each sequence to the longest in the batch.

#same order and same column name as in the dataframe 
fields = [ ('label', label_field),('sequence_splitted', text_field)]

# TabularDataset
train, valid, test = TabularDataset.splits(path='./data/', train='train2.csv', validation='valid2.csv',
                                            test='test2.csv',format='CSV', fields=fields, skip_header=True)
                                        

# Iterators
#each batch is of type torch.LongTensor, they are the numericalized batch
#BuketIterator Defines an iterator that batches examples of similar lengths together.Minimizes amount of 
#padding needed while producing freshly shuffled batches for each new epoch
train_iter = BucketIterator(train, batch_size=32, sort_key=lambda x: len(x.sequence_splitted),
                            device=device, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=32, sort_key=lambda x: len(x.sequence_splitted),
                            device=device, sort=True, sort_within_batch=True)
test_iter = BucketIterator(test, batch_size=1, device=device, sort=False, sort_within_batch=False, repeat=False)

"""
sort_within_batch must be flagged True if you want to use PyTorch’s padded sequence class.
repeat=None If you don’t want an epoch / iteration difference (epoch = a full traversal of your dataset, iteration = processed 
one batch from your dataset), and you want a simple while loop and a stop condition, then you should use repeat
It means that instead of iterating for one epoch, the iterator continues infinitely until stopped (in cases where you measure performance 
after # iterations and not # epochs).
The default will soon be changed to False (so the Uniterator will stop after one epoch).
use this so the loop will stop when training data is traversed once
epochs = 10

for epoch in range(epochs):
  for data in train_iter:
    (x, x_lengths), y = data.Text, data.Description
"""

# Vocabulary
text_field.build_vocab(train)

In [19]:
len(train_iter),len(valid_iter),len(test_iter)

(255, 64, 1800)

In [ ]:
#word count 
text_field.vocab.freqs

In [20]:
label_field.build_vocab(train)
label_field.vocab.freqs

Counter({'0': 1360, '4': 1360, '1': 1360, '5': 1360, '3': 1360, '2': 1360})

In [21]:
#enumerate labels
label_field.vocab.itos

['<unk>', '0', '1', '2', '3', '4', '5']

In [22]:
#encoding
label_field.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x2ae5a371dcd0>>,
            {'<unk>': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6})

## Model
Why pack sequences in pytorch explanation:
https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

In [23]:
########## DEFINE MODEL ########################
class LSTM(nn.Module):

    def __init__(self, dimension, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        #batch_size
        self.dimension = dimension 
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            batch_first=True,
                            bidirectional=bidirectional) #dropout=dropout if num_layers>1
        self.drop = nn.Dropout(p=dropout)
        #dense layer output from lstm bidirectional (batch_size, 2*batch_size)
        self.linear = nn.Linear(2*hidden_dim, 6) 

    def forward(self, text, text_len):
        #we pad the sequence for lst through embedding layer. In alternative we need to pad the sequence pad_packed_sequence
        text_emb = self.drop(self.embedding(text))
        #lstm_out, (ht, ct) = self.lstm(text_emb) ht=hidden ct=cell
        packed_input = pack_padded_sequence(text_emb, text_len.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (ht, ct) = self.lstm(packed_input)

        #unpack the output from lstm [dimension, 1, 2*dimension]      
        #output_lengths  is the length of each sentence
        output_padded, output_lengths= pad_packed_sequence(packed_output, batch_first=True)
       
        #get the foward and reverse lstm outputs, concatenate both of them and pass it to the fully connected layer.
        out_forward = output_padded[range(len(output_padded)), text_len - 1, :self.dimension]
        out_reverse = output_padded[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)

        text_fea = self.drop(out_reduced)
        
        text_fea = self.linear(text_fea)
        #remove size 1 [dimension,  #classes] 
        text_fea = torch.squeeze(text_fea, 1)
        
        
        #text_out=self.linear(ht[-1])

        #text_out=torch.nn.LogSoftmax(text_fea)
        return text_fea


In [24]:
# Save and Load Functions

def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    logging.info(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model, optimizer):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    logging.info(f'Model loaded from <== {load_path}')
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    logging.info(f'Model saved to ==> {save_path}')

def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    logging.info(f'Model loaded from <== {load_path}')
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

## Train and validation

In [30]:
def train(model,optimizer,criterion ,train_loader):
    running_loss = 0.0 
    global_step = 0 
    #sum_loss = 0.0
    total = 0.0
    correct = 0.0
    print('start training')
    #logging.info('start training'')
    model.train()       
    
    for (label,(text, text_len)), _  in train_loader: 
       
       if torch.cuda.is_available():
            label = label.to(device)
            text = text.to(device)
            text_len = text_len.to(device)
            output = model(text, text_len)
    
            loss_f = nn.CrossEntropyLoss()
            loss=loss_f(output, label.long())
            #loss = F.cross_entropy(output, label.long())
            #loss=criterion(output, label.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update train loss
            running_loss += loss.item()
            #train loss tentative 2   sum_loss/total,
            #sum_loss += loss.item()*label.shape[0]
            
            #train accuracy
            pred = torch.max(output, 1)[1]
            correct += (pred == label).float().sum()
            total += label.shape[0]
           
            #train acc
            #batch_acc = train_accuracy(output.cpu(), label.cpu())
            
            # count steps
            global_step += 1 
            if global_step % 50 == 0: 
                print(global_step)
                #logging.info(global_step)
    return running_loss, global_step, correct/total

In [29]:
def validate(model,criterion ,valid_loader):
    valid_running_loss = 0.0    
    correct = 0
    total = 0
    sum_rmse = 0.0
    sum_mae = 0.0
    sum_f1=0.0
    sum_prec=0.0
    sum_rec=0.0
    print('validate model')
    logging.info('validate model')
    #sum_loss = 0.0
    #batch_f1=0.0
    #batch_acc=0.0
    model.eval()
  
    with torch.no_grad():                    
        # validation loop
        for (label, (text, text_len)), _ in valid_loader:
          
           if torch.cuda.is_available():
                    label = label.to(device)
                    text = text.to(device)
                    text_len = text_len.to(device)
                    output = model(text, text_len)
                    #loss = criterion(output, labels)
                    
                    loss_f = nn.CrossEntropyLoss()
                    loss=loss_f(output, label.long())

                    #val loss
                    valid_running_loss += loss.item()
                    #val loss tentative 2
                    #sum_loss += loss.item()*label.shape[0]

                    pred = torch.max(output, 1)[1]
                    correct += (pred == label).float().sum()
                    total += label.shape[0]
                    
                    #METRICS

                    #pytorch_lightning not equal with sklearn, not sure about them TODO check
                    #batch_acc = valid_accuracy.update(output.cpu(), label.cpu())
                    #batch_f1 = f1.update(pred.cpu(), label.cpu())
                    #batch_precision=precision.update(output.cpu(), label.cpu())
                    #batch_recall=recall.update(output.cpu(), label.cpu())
                    #mean_absolute_error(output.cpu()*label.shape[0], label.long()*label.shape[0])
                    #mean_squared_error(output, label)

                    sum_rmse += np.sqrt(sklearn.metrics.mean_squared_error(label.unsqueeze(-1).cpu(),pred.cpu()))*label.shape[0]
                    sum_mae += sklearn.metrics.mean_absolute_error(label.unsqueeze(-1).cpu(),pred.cpu())*label.shape[0]
                    sum_f1 +=f1_score(label.unsqueeze(-1).cpu(),pred.cpu(), average='macro')*label.shape[0]
                    sum_prec +=precision_score(label.unsqueeze(-1).cpu(),pred.cpu(), average='macro', zero_division=0)*label.shape[0]
                    sum_rec +=recall_score(label.unsqueeze(-1).cpu(),pred.cpu(), average='macro', zero_division=0)*label.shape[0]
                    

    return valid_running_loss, correct/total, sum_rmse/total, sum_mae/total, sum_prec/total, sum_rec/total, sum_f1/total


In [32]:
def train_model(model, optimizer, criterion , train_loader , valid_loader , num_epochs , file_path , best_valid_loss):
    
    # initialize running values
    print('train_model')
    logging.info('train_model')
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    for epoch in range(num_epochs):
        print('train_model EPOCH '+str(epoch))        
        logging.info('train_model EPOCH '+str(epoch))
        train_loss,global_step, train_acc = train(model,  optimizer, criterion,train_loader)
     
        val_loss, val_acc, val_rmse, val_rmae, val_precision, val_recall, val_f1 = validate(model, criterion, valid_loader)
         
        # evaluation
        average_train_loss = train_loss / len(train_loader) 
        average_valid_loss = val_loss / len(valid_loader)
        train_loss_list.append(average_train_loss)
        valid_loss_list.append(average_valid_loss)
        global_steps_list.append(global_step)

        # print progress        
        print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader), average_train_loss, average_valid_loss))     
        print(" train acc %.3f, val accuracy %.3f, val rmse %.3f, val rmae %.3f, val precision %.3f, val recall %.3f, val f1 %.3f"
         % ( train_acc, val_acc, val_rmse, val_rmae, val_precision, val_recall, val_f1)) 
        logging.info('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader), average_train_loss, average_valid_loss))
        logging.info(" train acc %.3f, val accuracy %.3f, val rmse %.3f, val rmae %.3f, val precision %.3f, val recall %.3f, val f1 %.3f"
         % ( train_acc, val_acc, val_rmse, val_rmae, val_precision, val_recall, val_f1))
        #pytorch lightning
        #print("train acc %.3f, val accuracy %.3f" % (train_accuracy.compute(), valid_accuracy.compute()))  
        #print("f1 %.3f, precision %.3f, recall %.3f, mean_absolute_error %.3f" % (f1.compute(), precision.compute(),recall.compute(),val_rmae))      

    
        # resetting running values
        train_loss = 0.0                
        val_loss = 0.0        
        global_step = 0.0    
        val_acc = 0.0 
        val_rmse = 0.0 
        val_rmae = 0.0  
        val_precision = 0.0  
        val_recall = 0.0  
        val_f1 = 0.0  

        model.train()

        
        # checkpoint
        if best_valid_loss > average_valid_loss:
            best_valid_loss = average_valid_loss
            print(f'Best validation loss!! {best_valid_loss}')
            logging.info(f'Best validation loss!! {best_valid_loss}')
            save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
            save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
        save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
        print('Finished Training!')
        logging.info('Finished Training!')

## Evaluation

In [38]:
# Evaluation Function

def evaluate(model, test_loader,  threshold=0.5):

    correct = 0
    total = 0    
    sum_rmse = 0.0
    sum_mae = 0.0
    sum_f1=0.0
    sum_prec=0.0
    sum_rec=0.0
    sum_acc=0.0

    model.eval()

    confusion_matrix = torch.zeros(6,6)
   
    with torch.no_grad():
        for (labels, (text, text_len)), _ in test_loader:  
          
          if torch.cuda.is_available():         
            labels = labels.to(device)
            text = text.to(device)
            text_len = text_len.to(device)
            output = model(text, text_len)

            #output = (output > threshold).int()
            #y_pred.extend(output.tolist())
            #y_true.extend(labels.tolist())
            #y_pred=torch.max(output.data, 1)
            #print(y_pred)
            _, preds = torch.max(output, 1)
            
            for t, p in zip(labels.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            #preds = torch.max(output, 1)[1]

            correct += (preds == labels).float().sum()
            total += labels.shape[0]

            sum_rmse += sklearn.metrics.mean_squared_error(labels.unsqueeze(-1).cpu(),preds.cpu(),squared=False)*labels.shape[0]
            sum_mae += sklearn.metrics.mean_absolute_error(labels.unsqueeze(-1).cpu(),preds.cpu())*labels.shape[0]
            sum_f1 +=f1_score(labels.unsqueeze(-1).cpu(),preds.cpu(), average='macro')*labels.shape[0]
            sum_prec +=precision_score(labels.unsqueeze(-1).cpu(),preds.cpu(), average='macro', zero_division=0)*labels.shape[0]
            sum_rec +=recall_score(labels.unsqueeze(-1).cpu(),preds.cpu(), average='macro', zero_division=0)*labels.shape[0]
       
    print(confusion_matrix)
    print('Accuracy', correct/total)
    print('Precision', sum_prec/total)
    print('Recall', sum_rec/total)
    print('F1', sum_f1/total)
    print('RMSE', sum_rmse/total)
    print('MAE', sum_mae/total)
    logging.info(confusion_matrix)
    logging.info(f'Accuracy {correct/total}')
    logging.info(f'Precision {sum_prec/total}')
    logging.info(f'Recall {sum_rec/total}')
    logging.info(f'F1 {sum_f1/total}')
    logging.info(f'RMSE {sum_rmse/total}')
    logging.info(f'MAE {sum_mae/total}')
    return confusion_matrix

# Main

In [ ]:
def main():
    # print version and environment information
    print(f"{torch.__version__}")
    print(f"{torch.version.cuda}")
    print(f"{torch.backends.cudnn.enabled}")
    print(f"{torch.cuda.is_available()}")
    print(f"{device}")
    if torch.cuda.is_available():
        print(f"{torch.cuda.get_device_properties(device)}")
       
    
    destination_folder='data/'
    
    text_field, label_field, fields = load_fields()

    train, valid, test=load_tabular_dataset(fields)

    train_iter, valid_iter, test_iter= load_iterators(train, valid, test)

    # Vocabulary
    text_field.build_vocab(train)
    label_field.build_vocab(train)

    logging.info(label_field.vocab.freqs)

    #Define model
    model = LSTM(dimension=32, vocab_size=len(text_field.vocab), embedding_dim=150, hidden_dim=32, output_dim=6, n_layers=2, 
                 bidirectional=True, dropout=0.6, pad_idx=text_field.vocab.stoi[text_field.pad_token]).to(device)


    optimizer = optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    train_model(model,optimizer,criterion = nn.CrossEntropyLoss(),train_loader = train_iter,valid_loader = valid_iter,
          num_epochs = 30,file_path = destination_folder,
          best_valid_loss = float("Inf")) 
    logging.info("--- %s seconds ---" % (time.time() - start_time))



if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print()
        print("Interrupted with CTRL-C, exiting...")
        sys.exit()

In [35]:
########## SET MODEL PARAMETERS ########################
#n_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together 
#to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final 
        
#embedding_dim 100-300 even with a bigger vocabulary  

#dimension=batch size 
########################################################

model = LSTM(dimension=32, vocab_size=len(text_field.vocab), embedding_dim=120, hidden_dim=32, 
             output_dim=6, n_layers=1, 
                 bidirectional=True, dropout=0.3, pad_idx=text_field.vocab.stoi[text_field.pad_token]).to(device)     

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 50,134 trainable parameters


In [ ]:
#pytorch_lightening metrics
#train_accuracy = metrics.Accuracy()
#valid_accuracy = metrics.Accuracy()
#f1 = F1(num_classes=6,compute_on_step=True,average='macro')#’macro’ computes metric for each class and uniformly averages them
#precision = Precision(num_classes=6,average='macro')
#recall = Recall(num_classes=6,average='macro')
#mean_absolute_error = MeanAbsoluteError()
#mean_squared_error = MeanSquaredError()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
start_time = time.time()

train_model(model,optimizer,criterion = nn.CrossEntropyLoss(),train_loader = train_iter,valid_loader = valid_iter,
          num_epochs = 30,file_path = destination_folder,
          best_valid_loss = float("Inf")) 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#plot training and validation losses
train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
plt.plot( train_loss_list, label='Train')
plt.plot( valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#TODO load model and continue training
best_model = model.to(device)
optimizer = optim.Adam(best_model.parameters(), lr=0.001)

load_checkpoint(destination_folder + '/model.pt', best_model, optimizer)


start_time = time.time()

train_model(best_model,optimizer,criterion = nn.CrossEntropyLoss(),train_loader = train_iter,valid_loader = valid_iter,
          num_epochs = 10,file_path = destination_folder,
          best_valid_loss = float("Inf")) 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#from torchtext.data import Iterator
#test_iter = Iterator(test, batch_size=1, device=device, sort=False, sort_within_batch=False, repeat=False)

In [ ]:
best_model = model.to(device)
optimizer = optim.Adam(best_model.parameters(), lr=0.001)

load_checkpoint(destination_folder + '/model.pt', best_model, optimizer)
confusion_matrix=evaluate(best_model, test_iter)

In [ ]:
# Confusion matrix
ax= plt.subplot()
sns.heatmap(confusion_matrix, annot=True, ax = ax, cmap='Blues')

ax.set_title('Confusion Matrix')

ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')

In [ ]:
#Confusion matrix: percentage of  data represented in each quadrant
ax= plt.subplot()
sns.heatmap(confusion_matrix/confusion_matrix.sum(), annot=True,fmt='.2%',ax = ax, cmap='Blues')
ax.set_title('Confusion Matrix')

ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')

In [ ]:
# get predicted label
label_field.vocab.itos[y_pred]